### Data Source from Yahoo_Finance

In [71]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime

In [269]:
class data:
    def __init__(self, _ticker):
        self.ticker = yf.Ticker(_ticker)
        
    def get_Dailydata(self, _startdate, _enddate):
        basic_data = self.ticker.history(start = _startdate, end = _enddate)
        data = {
            'open': round(basic_data['Open'],2),
            'high': round(basic_data['High'],2),
            'low': round(basic_data['Low'],2),
            'close': round(basic_data['Close'],2),
            'volume': round(basic_data['Volume'],2)
        }
        df = pd.DataFrame(data)
        df = df.reset_index()
        df = df.rename(columns={'Date': 'date'})
        df['date'] = df['date'].astype(str)
        #df = df.iloc[1:].reset_index(drop = True)
        return df

In [270]:
# data source test
a = data("TSLA")
a.get_Dailydata('2015-01-01',datetime.date.today())

,date,open,high,low,close,volume
0,2014-12-31,44.62,45.14,44.45,44.48,11487500
1,2015-01-02,44.57,44.65,42.65,43.86,23822000
2,2015-01-05,42.91,43.30,41.43,42.02,26842500
3,2015-01-06,42.01,42.84,40.84,42.26,31309500
4,2015-01-07,42.67,42.96,41.96,42.19,14842000
...,...,...,...,...,...,...
1555,2021-03-08,600.55,620.13,558.79,563.00,51497000
1556,2021-03-09,608.18,678.09,595.21,673.58,67028000
1557,2021-03-10,700.30,717.85,655.06,668.06,60486700
1558,2021-03-11,699.40,702.50,677.18,699.60,36106300


In [248]:
m = a.get_Dailydata('2015-01-01' ,datetime.date.today())
m.loc[0]['date']

'2014-12-31'

### back test frame
1. buy_sell robot 2. wallet 3. back test robot

In [249]:
class trade_Robot:

# Only to execute buy or sell action, return remain capital, current holding position
# currentPosition 现有的持仓股数
# trade_robot 从wallet获得最新的持仓情况（现有现金，现有股票持仓），
#             从backtest kernel获得当前股价，
#             从策略集合获得当前的操作
# 买卖操作返回的是（持有现金，持仓数目，股票现价，操作名称）


    def __init__(self, _ticker):
        self.ticker = _ticker
        
    def get_Currentwallet(self, _currentPrice, _currentCapital, _currentPosition):
        self.currentCapital = _currentCapital
        self.currentPosition = _currentPosition
        self.currentPrice = _currentPrice
        
    def buy(self, volume):
        if volume % 100 != 0:
            print('volume must be the multiple of 100')
            return 'Trade failed'
        if self.capital_Check('buy', volume):
            self.currentCapital = self.currentCapital - volume*self.currentPrice
            self.currentPosition = self.currentPosition + volume
            return self.currentCapital, self.currentPosition, self.currentPrice, self.ticker, 'Buy'
        else:
            print('Trade failed')
            return self.no_Action()
        
    def sell(self, volume):
        if volume % 100 != 0:
            print('volume must be the multiple of 100')
            return 'Trade failed'
        if self.capital_Check('sell', volume):
            self.currentCapital = self.currentCapital + volume*self.currentPrice
            self.currentPosition = self.currentPosition - volume
            return self.currentCapital, self.currentPosition, self.currentPrice, self.ticker, 'Sell'
        else:
            print('Trade failed')
            return self.no_Action()
        
    def no_Action(self):
        return self.currentCapital, self.currentPosition, self.currentPrice, self.ticker, 'No Action'
    
    def capital_Check(self, action, volume):
        if action == 'buy':
            if self.currentCapital >= volume*self.currentPrice:
                return True
            else:
                print('Insufficient current capital')
                return False
        if action == 'sell':
            if self.currentPosition >= volume:
                return True
            else:
                print('Insufficient holding position')
                return False   

In [250]:
t = trade_Robot("AAPL")
t.get_Currentwallet(10, 10000, 400)

In [251]:
a = t.sell(400)
a

(14000, 0, 10, 'AAPL', 'Sell')

In [257]:
class wallet:
    
    # store the data of current bought tickers, 
    # current capital, current hoding position, current share value, historical data
    # for now, the wallet only allow single ticker
    
    def __init__(self, _ticker, _initialCapital, _startdate):
        self.initialCapital = _initialCapital
        self.startdate = datetime.datetime.strptime(_startdate,'%Y-%m-%d')
        self.startIndex = 0
        self.currentIndex = self.startIndex
        self.ticker = _ticker
        self.shares_Data = pd.DataFrame([],
             columns = ['date', 
                        'ticker',
                        'remainCapital',
                        'currentPosition',
                        'currentShareprice',
                        'currentSharesValue',
                        'currentTotalCapital'
                       ])

    def initialization(self):
        # set yesterday of the startday as the initial date 
        self.shares_Data = pd.DataFrame({
            'date': [(self.startdate - datetime.timedelta(days=1))], # format: datetime
            'ticker': [self.ticker],
            'remainCapital': [self.initialCapital],
            'currentPosition': [0],
            'currentShareprice': [0],
            'currentSharesValue': [0],
            'currentTotalCapital': [self.initialCapital]
        })
        self.currentIndex = self.startIndex
        return
    
    def update_Wallet(self, currentDate, updatedCapital, updatedPosition, currentPrice):
        self.currentIndex += 1
        currentDate = datetime.datetime.strptime(currentDate,'%Y-%m-%d')
        self.shares_Data.loc[self.currentIndex] = [
            currentDate,
            self.ticker,
            updatedCapital,
            updatedPosition,
            currentPrice,
            updatedPosition*currentPrice,
            updatedCapital + updatedPosition*currentPrice
        ]
        return
    
    def current_stat(self):
        # get current wallet stat
        data = self.shares_Data.loc[self.currentIndex][:]
        return data
    
    def historical_stat(self):
        data = self.shares_Data
        return data

In [258]:
a = wallet('AAPL', 10000, '2021-03-07')

In [259]:
a.initialization()
a.shares_Data

,date,ticker,remainCapital,currentPosition,currentShareprice,currentSharesValue,currentTotalCapital
0,2021-03-06,AAPL,10000,0,0,0,10000


In [263]:
a.update_Wallet('2021-03-10', 6000, 400, 40)

In [264]:
a.current_stat()

date                   2021-03-10 00:00:00
ticker                                AAPL
remainCapital                         6000
currentPosition                        400
currentShareprice                       40
currentSharesValue                   16000
currentTotalCapital                  22000
Name: 2, dtype: object

In [265]:
a.historical_stat()

,date,ticker,remainCapital,currentPosition,currentShareprice,currentSharesValue,currentTotalCapital
0,2021-03-06,AAPL,10000,0,0,0,10000
1,2021-03-09,AAPL,6000,400,25,10000,16000
2,2021-03-10,AAPL,6000,400,40,16000,22000


In [266]:
a.initialization()
a.historical_stat()

,date,ticker,remainCapital,currentPosition,currentShareprice,currentSharesValue,currentTotalCapital
0,2021-03-06,AAPL,10000,0,0,0,10000


In [293]:
class backtest:
    
    # backtest kernel only got basic stock data (data, price) to execute trade and get simulated gain
    # strategy get the date info
    # trade get the action from strategy, get the current holding stat from wallet
    # wallet get the update holding stat from the trade
    
    def __init__(self, _ticker, _strategy, _startDate, _endDate):
        self.ticker = _ticker
        self.ds = data(_ticker)
        self.strategy = _strategy   
        self.initialCapital = self.strategy.initialCapital #get initial capital from strategy class
        self.startDate = _startDate
        self.startIndex = 0
        self.endIndex = 0
        self.wallet = wallet(_ticker, self.initialCapital, _startDate)
        self.wallet.initialization()  # initialize wallet at the beginning
        self.hist = self.ds.get_Dailydata(_startDate, _endDate) # get all historical basic data
        
    def execute(self):
        # loop: read operations from strategy, execute trade, update wallet and go to next trading day
        self.initializeKernel()
        print('Back test Start, initial wallet stat:')
        print(self.wallet.current_stat())
        print('-------------------------------------------')
        
        while True:
            isEnd, data = self.getABatch()
            if isEnd == True:
                print('Last Day Reached, End')
                break
            else:
                print('-----------------' + data['date'] + '-----------------')
                # transfer current wallet stat, current date to strategy, 
                st = self.strategy.get_Option(data['date'])
                
                # tell trade robot the trade action, return 
                
                # update wallet
                
                # update index, and go to next day
                
                self.seeU_Tomorrow()
            
        return
    
    def initializeKernel(self):
        # restart
        self.currentDate = self.startDate
        self.currentIndex = self.startIndex
        self.endIndex = self.startIndex + len(self.hist) # endIndex is 1 bigger than the last index
        self.remainHistdata = self.hist
        self.wallet.initialization() 
        return
    
    def getABatch(self):
        # same to getABatch (in old version)
        if self.currentIndex < self.endIndex:
            dataBatch = self.hist.loc[self.currentIndex][:]
            isEnd = False
            return isEnd, dataBatch
        else:
            isEnd = True
            return isEnd, []
    
    def seeU_Tomorrow(self):
        self.currentIndex += 1
        return
    
    def legal_Check(self):
        return
        

In [294]:
st = strategy_01(10000)
m = backtest("TSLA", st, '2021-03-01', datetime.date.today())

In [295]:
m.hist

,date,open,high,low,close,volume
0,2021-03-01,690.11,719.00,685.05,718.43,27009700
1,2021-03-02,718.28,721.11,685.00,686.44,23617600
2,2021-03-03,687.99,700.70,651.71,653.20,29957200
3,2021-03-04,655.80,873.94,600.00,621.44,65448500
4,2021-03-05,626.06,627.84,539.49,597.95,89300300
5,2021-03-08,600.55,620.13,558.79,563.00,51497000
6,2021-03-09,608.18,678.09,595.21,673.58,67028000
7,2021-03-10,700.30,717.85,655.06,668.06,60486700
8,2021-03-11,699.40,702.50,677.18,699.60,36106300
9,2021-03-12,670.00,694.88,666.14,693.73,33523900


In [296]:
f = m.hist.loc[5][:]
f['date']

'2021-03-08'

In [297]:
m.execute()

Back test Start, initial wallet stat:
date                   2021-02-28 00:00:00
ticker                                TSLA
remainCapital                        10000
currentPosition                          0
currentShareprice                        0
currentSharesValue                       0
currentTotalCapital                  10000
Name: 0, dtype: object
-------------------------------------------
param transit success
current date:2021-03-01
-----------------2021-03-01-----------------
param transit success
current date:2021-03-02
-----------------2021-03-02-----------------
param transit success
current date:2021-03-03
-----------------2021-03-03-----------------
param transit success
current date:2021-03-04
-----------------2021-03-04-----------------
param transit success
current date:2021-03-05
-----------------2021-03-05-----------------
param transit success
current date:2021-03-08
-----------------2021-03-08-----------------
param transit success
current date:2021-03-09
--

In [210]:
m.wallet.current_stat()

date                   2021-02-28 00:00:00
ticker                                TSLA
remainCapital                        10000
currentPosition                          0
currentShareprice                        0
currentSharesValue                       0
currentTotalCapital                  10000
Name: 0, dtype: object

In [292]:
# future strategy related to ML model will gain a external csv sheet (trained output) as the data
# linear model will use some same modes in this kind of strategy kits

class strategy_01:
    def __init__(self, _capitalAmount):
        self.initialCapital = _capitalAmount
        #self.target = _targetPrice
        #self.x = _param1
        #self.y = 2*_param1/(2*_param1 + 3)
        #self.stopLoss = self.target*(self.x + 1)*(1 - self.y)
        #self.boughtIndex = 1
        #self.ds = []
    
    def get_Option(self, currentDate):
        ###todayPrice = ds[currentDate]
        #print('param transit success')
        #print('current date:' + currentDate)
        return 'wow'
        
    def set_Buyline(self):
        return
    
    def set_Stoploss(self):
        return

In [85]:
s = strategy_01(1000)
s.initialCapital

1000